# Chapter 17: Iterators, Generators,and Classic Coroutines

## A sequence of Words

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 1
    
    def __getitem__(self, index):
        return self.words[index]  # 2
    
    def __len__(self):
        return len(self.words) # 3
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 4

1. `.findall` returns a list of all non-overlapping matches of pattern , as a list of strings. 
2. `self.words` holds the result of `findall` method, so we simply return the word at the given index.
3. To complete the sequence protocol, we implement `__len__` although it is not really needed for our purpose.
4. `reprlib.repr` limits the generated string to 30 characters. 

Testing iteration on a `Sentence` instance:
1. A sentence is created from a string.
2. Note the output of `__repr__` using ... generated by `reprlib.repr`.
3. `Sentence` instances are iterable; we'll see why in a moment.
4. Being iterable, `Sentence` objects can be used as input to build lists and other iterable types.

In [ ]:
s = Sentence('"The time has come," the Walrus said,')
s

In [ ]:
for word in s:
    print(word)

In [ ]:
list(s)

In [ ]:
words = Sentence('This is a test')
iterator = iter(words)
print(next(iterator))

## Why Sequences Are Iterable: The iter Function

## Sentence Take #2: A Classic Iterator

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
        
        def __init__(self, text):
            self.text = text
            self.words = RE_WORD.findall(text)
            
        def __repr__(self):
              return f'Sentence({reprlib.repr(self.text)})'
          
        def __iter__(self):
            return SentenceIterator(self.words)
        
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words
        self.index = 0
        
    def __next__(self):
        try:
            word = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

In [ ]:
words = Sentence('This is a test')
iterator = iter(words)
print(next(iterator))

## Sentence Take #4: Lazy Generator

Sentence implementation #1-#3 build a list of all words in the text, binding it to `self.words` attribute. This requires processing the entire text, and list may as much memory as the text itself. It's not lazy enough.

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text #1
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        for match in RE_WORD.finditer(self.text): #2
            yield match.group()  #3

In [ ]:
words = Sentence('This is a test')
iterator = iter(words)
print(next(iterator))

1. No need to have a word list.
2. `finditer` builds an iterator over the matches of `RE_WORD` on `self.text`, yielding `MatchObject` instance.
3. `match.group()` extracts the matched text from the `MatchObject` instance.

## Sentence Take #5: Lazy Generator Expression

We can replace simply generator function with a generator expression.

In [ ]:
import re
import reprlib

RE_WORD = re.compile(r'\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text #1
        
    def __repr__(self):
        return f'Sentence({reprlib.repr(self.text)})'
    
    def __iter__(self):
        return (match.group() for match in RE_WORD.finditer(self.text)) #2

In [ ]:
words = Sentence('This is a test')
iterator = iter(words)
print(next(iterator))

## When to Use Generator Expressions

A generator expression:

-  is a syntactic shortcut to create a generator without defining and calling a function.
-  is more flexible: we can code complex logic with multiple statements, and we can even use them as coroutines.

If the generator expression spans more than a couple of lines, I prefer to code a generator function for the sake of readability.

## Contrasting Iterators and Generators

- iterator: General term for any object that implements a `__next__` method. Iterators are designed to produce data that is consumed by the client data. In practice, most iterators we use in Python are `generators`.
- generator: An iterator built by the Python compiler. To create a generator, we don't implement `__next__`. Instead, we use the `yield` keyword to make a generator function, which is a factory of generator objects. A generator expression is another way to build a generator object. Generator objects provide `__next__`, so they are iterators. Since Python 3.5, we also have a asynchronous generators declared with `async def`. 

In [ ]:
def g():
    yield 0

ge = (c for c in 'XYZ')
g(), ge

In [ ]:
type(g()), type(ge)

In [ ]:
print(dir(g()))

## Rewrite Sentence Class to a Generator Function

In [ ]:
import re

RE_WORD = re.compile(r'\w+')

def sentence(text):
    for match in RE_WORD.finditer(text):
        yield match.group()

In [ ]:
s

In [ ]:
msg = 'Hello Fluent Python'
s = sentence(msg)
print(next(s))

## An Arithmetic Progression Generator

The `range` builtin generates a bounded arithmetic progression(AP) of integers.

In [ ]:
class ArithmeticProgression:
    
    def __init__(self, begin, step, end=None): #1
        self.begin = begin
        self.step = step
        self.end = end # None -> "infinite" series
        
    def __iter__(self):
        result_type = type(self.begin + self.step) #2
        result = result_type(self.begin) #3
        forever = self.end is None #4
        # end = result_type(self.end) if not forever else end 
        index = 0
        while forever or result < self.end: #5
            yield result  #6
            index += 1
            result = self.begin + self.step * index #7

1. `__init__` requires two arguments: `begin` and `step`; `end` is optional, if it's `None`, the series will be unbounded.
2. Get the type of adding `self.begin` and `self.step`. For example, if one is `int` and the other is `float`, `result_type` will be `float`.
3. This line makes a `result` with the same numeric value of `self.begin` but coerced to `result_type`.
4. For readability, the `forever` flag will be `True` if `end` is `None`, resulting in an unbounded series.
5. This loop runs `forever` or until `result` exceeds `self.end`. When this loop exits, so does the generator.
6. The current `result` is produced.
7. The next potential `result` is calculated. It may never be yielded, if the loop ends here.

In [ ]:
ap = ArithmeticProgression(0, 1, 3)
print(list(ap))

ap = ArithmeticProgression(1, .5, 3)
print(list(ap))

ap = ArithmeticProgression(0, 1/3, 1)
print(list(ap))

In [ ]:
# Doesn't support complex numbers if end is not None
# TypeError: '<' not supported between instances of 'complex' and 'complex'
# Are there any ways to compare complex numbers?

ap = ArithmeticProgression(1, 0.5+.5j)
iterator = iter(ap)
for _ in range(10):
    print(next(iterator))

Maybe we don't need a class here, all we need is a function.
A generator object is created with `__next__` and `__iter__` methods.
That's much better than a class object only with `__iter__` method but without `__next__` method.


In [ ]:
def arithmetic_progression(begin, step, end=None):
    result_type = type(begin + step)
    result = result_type(begin)
    forever = end is None
    index = 0
    while forever or result < end:
        yield result
        index += 1
        result = begin + step * index

In [ ]:
ap = arithmetic_progression(0, 1, 3)
print(next(ap))
print(list(ap))

## Arithmetic Progression with itertools

In [ ]:
import itertools
gen = itertools.count(1, .5)
print(next(gen))
print(next(gen))
print(next(gen))

In [ ]:
import itertools
gen = itertools.count(1, .5j)
print(next(gen))
print(next(gen))
print(next(gen))

`itertools.count` never stops, so if you call `list(count())`, Python will try to build a `list` that would fill all available memory and crash the program.

On the other hand, there is the `itertools.takewhile` function, which also produces a generator, but consumes another generator or iterable to stop after a condition evaluates to `False`.

In [ ]:
gen = itertools.takewhile(lambda n: n < 3, itertools.count(1, .5))
print(list(gen))

## Generator Functions in the Standard Library

### Table 17.1 Filtering generator functions

| Module | Function | Description |
| --- | --- | --- |
| itertools | `takewhile(predicate, it)` | consumes a generator and stops at a condition |
| itertools | `dropwhile(predicate, it)` | consumes a generator and drops items while a condition holds |
| itertools | `compress(it, selector_it)` | consumes a generator and an iterable, returning only the items from the iterable for which the corresponding item in the generator is truthy |
| builtin | `filter(predicate, it)` | consumes a function and an iterable, returning only the items from the iterable for which the function returns truthy |
| itertools | `filterfalse(predicate, it)` | consumes a generator and an iterable, returning only the items from the iterable for which the corresponding item in the generator is falsy |
|itertools | `islice(it, stop)` or `islice(it, start, stop, step=1)` | consumes a generator and returns an iterator that produces selected items from the original generator, by index |

In [ ]:
vowel = lambda c: c.lower() in 'aeiou'

print(list(filter(vowel, 'Aardvark')))

In [ ]:
import itertools

print(list(itertools.filterfalse(vowel, 'Aardvark')))
print(list(itertools.dropwhile(vowel, 'Aardvark')))
print(list(itertools.takewhile(vowel, 'Aardvark')))
print(list(itertools.compress('Aardvark', (1,0,1,1,0,1))))
print(list(itertools.islice('Aardvark', 4)))
print(list(itertools.islice('Aardvark', 4, 7)))
print(list(itertools.islice('Aardvark', 1, 7, 2)))

### Table 17.2 Mapping generator functions

| Module | Function | Description |
| --- | --- | --- |
| itertools | `accumulate(it, [func])` | consumes a generator and returns an iterator that produces accumulated sums, or accumulated results of other binary functions |
| itertools | `starmap(func, it)` | consumes a generator and an iterable, returning an iterator that produces the result of passing the items from the iterable to the function as individual arguments |
| builtin | `map(func, it)` | consumes a function and an iterable, returning an iterator that produces the result of passing the items from the iterable to the function as individual arguments |
| builtin | `enumerate(it, start=0)` | consumes an iterable and returns an iterator that produces tuples of `(index, item)` pairs, where `index` starts at `start` and `item` are the values from the iterable |

In [ ]:
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]
import itertools

print(list(itertools.accumulate(sample)))
print(list(itertools.accumulate(sample, min)))
print(list(itertools.accumulate(sample, max)))

import operator
print(list(itertools.accumulate(sample, operator.mul)))
print(list(itertools.accumulate(range(1, 11), operator.mul)))

In [ ]:
print(list(enumerate('albatroz', 1)))

import operator
print(list(map(operator.mul, range(11), range(11))))
print(list(map(operator.mul, range(11), [2, 4, 8])))
print(list(map(lambda a, b: (a, b), range(11), [2, 4, 8])))

In [ ]:
import itertools

def add(x, y, z):
    return x + y + z

iterable = [(1, 2, 3), (3, 4, 5), (5, 6, 7)]

result = list(itertools.starmap(add, iterable))
print(result)  # Output: [6, 12, 18]

In [ ]:
import itertools
print(list(itertools.starmap(operator.mul, enumerate('albatroz', 1))))
sample = [5, 4, 2, 8, 7, 6, 3, 0, 9, 1]

# average
print(list(itertools.starmap(lambda a, b: b/a, enumerate(itertools.accumulate(sample), 1))))

### Table 17.3 Merging generator functions

| Module | Function | Description |
| --- | --- | --- |
| itertools | `chain(it1, ..., itN)` | consumes multiple iterables in order, yielding their items one after the other |
| itertools | `chain.from_iterable(it)` | consumes a single iterable of iterables, yielding their items one after the other; `it` will be an iterable where the items are also iterables, for example, a list of tuples. |
| itertools | `product(it1, ..., itN, repeat=1)` | cartesian product; yields N-tuples made by combining items from each input iterable like nested for loops; `repeat` allows the input iterables to be consumed more than once |
| builtin | `zip(it1, ..., itN, strict=False)` | consumes multiple iterables in parallel, yielding tuples made from their respective items, stopping when the first iterable is exhausted, unless `strict=True` is given |
| itertools | `zip_longest(it1, ..., itN, fillvalue=None)` | consumes multiple iterables in parallel, yielding tuples made from their respective items, stopping when the last iterable is exhausted, filling missing values with `fillvalue` |

In [ ]:
print(list(itertools.chain('ABC', range(2))))

In [ ]:
print(list(itertools.chain(enumerate('ABC'))))

`chain.from_iterable` takes each item from iterable, and chains them together into a single sequence, as long as each item is itself iterable. 

In [ ]:
print(list(itertools.chain.from_iterable(enumerate('ABC'))))

In [ ]:
print(list(zip('ABC', range(5), [10, 20, 30, 40, 50])))

In [ ]:
list(itertools.zip_longest('ABC', range(5)))

In [ ]:
list(itertools.zip_longest('ABC', range(5), fillvalue='?'))

`itertools.product` generator function examples

In [ ]:
list(itertools.product('ABC', range(2)))

In [ ]:
suits = 'spades hearts diamonds clubs'.split()
list(itertools.product('AK', suits))

In [ ]:
list(itertools.product('ABC'))

In [ ]:
list(itertools.product('ABC', repeat=2))

In [ ]:
list(itertools.product(range(2), repeat=3))

In [ ]:
rows = list(itertools.product('AB', range(2), repeat=2))
for row in rows:
    print(row)

### Table 17.4 Generator functions that expand each input item into multiple output items

| Module | Function | Description |
| --- | --- | --- |
| itertools | `combinations(it, out_len)` | consumes an iterable and yields tuples of `out_len` items from it, without replacement |
| itertools | `combinations_with_replacement(it, out_len)` | consumes an iterable and yields tuples of `out_len` items from it, with replacement |
| itertools | `count(start=0, step=1)` | produces a generator that returns numbers starting with `start`, incremented by `step`, indefinitely |
| itertools | `cycle(it)` | consumes an iterable and produces an iterator that repeats the items indefinitely |
| itertools | `permutations(it, out_len=None)` | consumes an iterable and yields tuples of `out_len` items from it, with replacement; by default, `out_len` is `len(list(it))` |
| itertools | `repeat(item, [times])` | produces a generator that yields `item` over and over, indefinitely unless `times` is given |
| itertools | `pairwise(it)` | yield successive overlapping pairs of items from `it`; for example, `pairwise('ABC')` yields `('A', 'B')`, then `('B', 'C')` |

count, cycle, pairwise, and repeat examples.

In [ ]:
ct = itertools.count()
next(ct), next(ct), next(ct)

In [ ]:
list(itertools.islice(itertools.count(1, .3), 3))

In [ ]:
cy = itertools.cycle('ABC')
next(cy), next(cy), next(cy), next(cy)

In [ ]:
list(itertools.islice(cy, 7))

In [ ]:
list(itertools.pairwise(range(7)))

In [ ]:
rp = itertools.repeat(7)
next(rp), next(rp)

In [ ]:
list(itertools.repeat(8, 4))

In [ ]:
list(map(operator.mul, range(11), itertools.repeat(5)))

Combinatoric generator functions yield multiple values per input item

In [ ]:
list(itertools.combinations('ABC', 2))

In [ ]:
list(itertools.combinations_with_replacement('ABC', 2))

In [ ]:
list(itertools.permutations('ABC', 2))

In [ ]:
list(itertools.product('ABC', repeat=2))

### Table 17.5 Rearranging generator functions

| Module | Function | Description |
| --- | --- | --- |
| itertools | `groupby(it, key=None)` | yield 2-tuples of the form(key, group), where key is the grouping criterion and group is generator yielding the items in the group |
| itertools | `tee(it, n=2)` | consumes an iterable and returns `n` independent iterators, each yielding the items of the original iterable; if `n` is not specified, it defaults to 2 |
| builtin | `reversed(seq)` | consumes a sequence and returns an iterator that yields items from the sequence in reverse order |

In [ ]:
import itertools
list(itertools.groupby('LLLLAAGGG'))

In [ ]:
for char, group in itertools.groupby('LLLLAAAGG'):
    print(char, '->', list(group))

In [ ]:
animals = ['duck', 'eagle', 'rat', 'giraffe', 'bear', 
        'bat', 'dolphin', 'shark', 'lion']

animals.sort(key=len)
animals

In [ ]:
for length, group in itertools.groupby(animals, len):
    print(length, '->', list(group))

In [ ]:
for length, group in itertools.groupby(reversed(animals), len):
    print(length, '->', list(group))

In [ ]:
list(itertools.tee('ABC'))

In [ ]:
g1, g2 = itertools.tee('ABC')
next(g1)

In [ ]:
next(g2)

In [ ]:
next(g2)

In [ ]:
list(g1), list(g2)

In [ ]:
list(zip(*itertools.tee('ABC')))

### Table 17.6 Iterable Reducing Functions

| Module | Function | Description |
| --- | --- | --- |
| builtin | `all(it)` | consumes an iterable and returns `True` if all items are truthy, `False` otherwise |
| builtin | `any(it)` | consumes an iterable and returns `True` if any item is truthy, `False` otherwise |
| builtin | `max(it, [key=,] [default=])` | consumes an iterable and returns the largest item, optionally using `key` function and `default` value |
| builtin | `min(it, [key=,] [default=])` | consumes an iterable and returns the smallest item, optionally using `key` function and `default` value |
| builtin | `sum(it, start=0)` | consumes an iterable and returns the sum of items, starting with `start` |
| itertools | `reduce(func, it, [initial=])` | consumes an iterable and returns a single value constructed by calling the `func` function on the first two items, then on the result and the next item, and so on; if `initial` is given, it is placed before the items of the iterable in the calculation, and serves as a default when the iterable is empty |

In [ ]:
all([1, 2, 3])

In [ ]:
all([1, 0, 3])

In [ ]:
all([])

In [ ]:
any([1, 2, 3])

In [ ]:
any([1, 0, 3])

In [ ]:
any([0, 0, 0])

In [ ]:
any([])

In [ ]:
# any() and all() are short-circuiting,
# any iterated over g until yielded a True value that is 7,
# then any stopped and returned True
g = iter([0, 0.0, 7, 8])
any(g)

In [ ]:
# That is why 8 was still remaining in g
next(g)

## Subgenerators with yield from

The `yield from` expression syntax was introduced in Python 3.3 to allow a generator to delegate work to a subgenerator.

Before `yield from`, the only way to do that was to write a for loop when a generator needed to yield values produced from another generator.

In [ ]:
def sub_gen():
    yield 1
    yield 2
    yield 3
    
def gen():
    yield 'A'
    yield 'B'
    for i in sub_gen():
        yield i
    yield 'C'
    
for x in gen():
    print(x)

In [ ]:
def sub_gen():
    yield 1
    yield 2
    yield 3
    
def gen():
    yield 'A'
    yield 'B'
    yield from sub_gen()
    yield 'C'
    
for x in gen():
    print(x)

In this example, the for loop is the client code, `gen` is the **delegating generator**, and `sub_gen` is the **subgenerator**. Note that `yield from` pauses `gen` and `sub_gen` takes over until it is exhausted. The values yielded by `sub_gen` are passed directly to `gen`'s caller.

When the subgenerator contains a `return` statement with a value, that value can be captured in the delegating generator by using `yield from` as part of an expression.

In [ ]:
def sub_gen():
    yield 1.1
    yield 2.2
    return 'Done!'

def gen():
    yield 1
    result = yield from sub_gen()
    print('sub_gen returned:', result)
    yield 2
    
for x in gen():
    print(x)

### Reinventing chain

In [ ]:
def chain(*iterables):
    for it in iterables:
        yield from it
        
s = 'ABC'
t = tuple(range(3))
list(chain(s, t))

### Traversing a Tree

We'll see `yield from` in a script to traverse a tree structure.

The tree structure for this example is Python's `exception hierarchy`. The `BaseException` class is the root of the hierarchy, and all other exception classes are its descendants, the exception hierarchy is five levels deep as of Python 3.10.

In [ ]:
# Yield the name of the root class and stop
def tree(cls):
    yield cls.__name__
    
def display(cls):
    for cls_name in tree(cls):
        print(cls_name)
        
if __name__ == '__main__':
    display(BaseException)

In [ ]:
# Yield the name of the root class and direct subclasses
def tree(cls):
    yield cls.__name__, 0
    for subclass in cls.__subclasses__():
        yield subclass.__name__, 1
        
def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')
        
if __name__ == '__main__':
    display(BaseException)

In [ ]:
# Yields the root class name, then delegate to sub_tree,
# recursive sub_tree goes as far as memory allows
def tree(cls):
    yield cls.__name__, 0
    yield from sub_tree(cls, 1)
    
def sub_tree(cls, level):
    for subclass in cls.__subclasses__():
        yield subclass.__name__, level
        yield from sub_tree(subclass, level+1)
        
def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')
        
if __name__ == '__main__':
    display(BaseException)



In [ ]:
# Recursive calls of tree pass an incremental level argument
def tree(cls, level=0):
    yield cls.__name__, level
    for subclass in cls.__subclasses__():
        yield from tree(subclass, level+1)
        
def display(cls):
    for cls_name, level in tree(cls):
        indent = ' ' * 4 * level
        print(f'{indent}{cls_name}')
        
if __name__ == '__main__':
    display(BaseException)

## Generic Iterable Types

In [1]:
from collections.abc import Iterable

FromTo = tuple[str, str] #1

def zip_replace(text:str, changes: Iterable[FromTo]) -> str: #2
    for src, dest in changes:
        text = text.replace(src, dest)
    return text

1. Define type alias; not required, but makes the next hint more readable. Starting with Python 3.10, `FromTo` should have a type hint of `typing.TypeAlias` to clarify the reason for this line.
2. Annotate `changes` to accept an `Iterable` of `FromTo` tuples.

In [2]:
msg = 'hello, world!'
changes = [('hello', 'goodbye'), ('world', 'earth')]
print(zip_replace(msg, changes))

goodbye, earth!


### Two ways to annotate iterators

In [8]:
from collections.abc import Iterator
from keyword import kwlist
from typing import TYPE_CHECKING

short_kw = (k for k in kwlist if len(k) < 5) #1

if TYPE_CHECKING:
    reveal_type(short_kw) #2

long_kw: Iterator[str] = (k for k in kwlist if len(k) >= 4) #3

if TYPE_CHECKING:
    reveal_type(long_kw) #4

In [7]:
print(list(short_kw))

['None', 'True', 'and', 'as', 'def', 'del', 'elif', 'else', 'for', 'from', 'if', 'in', 'is', 'not', 'or', 'pass', 'try', 'with']


1. Generator expression that yields Python keywords with less than 5 characters.
2. Mypy infers: `typing.Generator[str, None, None]`.
3. This also yields strings, but I added an explicit hint.
4. Mypy infers: `typing.Iterator[str]`.

## Classic Coroutines

A coroutine is really a generator function, created with the `yield` in its body. And a coroutine object is physically a generator object. But the use case of generators and coroutines are very different in Python.

The formal type parameters of `Generator` like this:

```python
Generator[YieldType, SendType, ReturnType]
```

The `SendType` is only relevant when the generator is used as a coroutine. That type parameter is the type of `x` in the call `gen.send(x)`. It is an error to call `.send()` on a generator that was coded to behave as an iterator instead of a coroutine. Likewise, `ReturnType` is only meaningful to annotate a coroutine, because iterator don't return values like regular functions. The only sensible operation on a generator used as an iterator is to call `next(it)` or indirectly via `for` loop and other forms of iteration. The `YieldType` is the type of the value returned by a call to `next(it)`.

-  Generators produce data for iteration.
-  Coroutines are consumer of data.
-  To keep your brain from exploding, don't mix the two concepts together.
-  Coroutines are not related to iteration.
-  Note: There is a use of having `yield` produce a value in a coroutine, but it's not tied to iteration.

### Example: Coroutine to Compute a Running Average

In [23]:
from collections.abc import Generator

# def averager() -> Generator[float, float, None]:
def averager():
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield average
        total += term
        count += 1
        average = total/count    

In [28]:
coro_avg = averager()
next(coro_avg)

0.0

In [29]:
coro_avg.send(10)

10.0

In [30]:
coro_avg.send(30)

20.0

In [31]:
coro_avg.send(5)

15.0

In [32]:
coro_avg.close()

In [33]:
coro_avg.send(10)

StopIteration: 

In [9]:
def my_coroutine():
    term1 = yield 5
    term2 = yield 8 + term1
    term3 = yield 15 - term2
    yield term3

my_coro = my_coroutine()
print(next(my_coro))
print(my_coro.send(1))
print(my_coro.send(2))
print(my_coro.send(3))

5
9
13
3


In [4]:
def my_coroutine():
    result = 0
    while True:
        term = yield result
        result = term  

my_coro = my_coroutine()
print(next(my_coro))
print(my_coro.send(10))
print(my_coro.send(20))

0
10
20


### Returning a Value from a Coroutine

In [12]:
from collections.abc import Generator
from typing import Union, NamedTuple

class Result(NamedTuple):
    count: int
    average: float
    
class Sentinel:
    def __repr__(self) -> str:
        return '<Sentinel>'
    
STOP = Sentinel()

SendType = Union[float, Sentinel]

In [46]:
def averager2(verbose: bool = False) -> Generator[SendType, float, Result]:
    total = 0.0
    count = 0
    average = 0.0
    while True:
        term = yield
        if verbose:
            print('received:', term)
        if isinstance(term, Sentinel):
            break
        total += term
        count += 1
        average = total / count
    return Result(count, average)

In [47]:
coro_avg = averager2(verbose=True)
print(next(coro_avg))

None


In [48]:
coro_avg.send(10)

received: 10


In [49]:
coro_avg.send(20)

received: 20


In [50]:
coro_avg.send(10)

received: 10


In [51]:
try:
    coro_avg.send(STOP)
except StopIteration as exc:
    result = exc.value

print(result)

received: <Sentinel>
Result(count=3, average=13.333333333333334)


In [45]:
def compute():
    res = yield from averager2(True)
    print('computed:', res)
    return res

comp = compute()
for v in [None, 10, 20, 30 , STOP]:
    try:
        comp.send(v)
    except StopIteration as exc:
        result = exc.value   

received: 10
received: 20
received: 30
received: <Sentinel>
computed: Result(count=3, average=20.0)


### Generic Type Hints for Classic Coroutines

A `Generator` type hint requires three type parameters:
`my_coro : Generator[YieldType, SendType, ReturnType]`

From the type variables in the formal parameters, we see `YieldType` and `ReturnType` are covariant, and `SendType` is contravariant. To understand why, consider `YieldType` and `ReturnType` are "output" types. Both describe data that comes out of the coroutine--i.e., the generator object when used as a coroutine object.

`SendType` is an input parameter: if a formal type parameter defines a type for data that goes into the object after its initial construction, it can be contravariant.